___
# Projeto 2 - Ciência dos Dados 2020.2
Grupo: Lucas Kang
___

# Descrição do projeto

O projeto foi inspirado na ideia de observar o relacionamento entre vários indicadores como a idade, sexo, situação econômica do país, e a geração, com o número de suicídios de um paí

## Objetivo do projeto:

- A partir de diversos fatores como o ano, sexo, faixa etária e situação econômica de um país, é possível prever a quantidade de pessoas que irão cometer suicídio neste ano?  
- O target é quantitativo, já que o objetivo é prever uma informação numérica.

In [32]:
#importando as bibliotecas a serem utilizadas
import pandas as pd  
import matplotlib.pyplot as plt
import numpy as np
from math import *
from scipy import *
from sklearn import linear_model
from scipy import stats

In [2]:
#importando a base de dados a ser utilizada
suicidios = pd.read_csv('suicide_rates.csv')

# apagar depois

## coisas a serem estudadas

- multiple linear regression
- random forest regression
- variável qualitativa precisa ser transformada em dummies
- pd.get_dummies() -- estudar e ler a documentação do comando

In [3]:
suicidios.head(5)

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


# Organizando os dados

Os dados foram organizados em novas colunas e alguns valores foram traduzidos para números que possa ser utilizados pelo modelo para realizar a predição.
Algumas destas novas colunas foram:
- criação de dummies para armazenar a faixa etária de cada linha;
- criação da coluna de classificação econômica;
- traduzir a coluna de geração de valores categóricos para valores numéricos;
- traduzir a coluna de sexo de valores categóricos para valores numéricos.

In [4]:
#criação de uma dummies para armazenar a faixa etária em formato numérico
faixa_etaria = pd.get_dummies(suicidios['age'])
suicidios = pd.concat([suicidios, faixa_etaria], axis = 1)
suicidios.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation,15-24 years,25-34 years,35-54 years,5-14 years,55-74 years,75+ years
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X,1,0,0,0,0,0
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent,0,0,1,0,0,0
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X,1,0,0,0,0,0
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation,0,0,0,0,0,1
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers,0,1,0,0,0,0


In [5]:
#criação de uma nova coluna para armazenar a faixa etária em formato numérico
#suicidios['faixa_etaria'] = suicidios['age'].astype('int64')
#suicidios['faixa_etaria'].int64 = [1, 2, 3, 4, 5, 6]

In [7]:
# comparar gdp_per_capita pra determinar se o país naquele ano era considerado Rico, Em desenvolvimento, ou Pobre (e.g: resultado maior que 30000 dol = rico, menor que 5000 dol/pessoa = probre)
# Utilizando a base de dados do IMF, foi feito uma divisão do gdp do país pelo gdp de Luxemburgo do ano de 2020 (USD 109602). 
# Os países que se encontraram com porcentagem abaixo de 30% foram consideradas pobres, de 31% a 60%, em desenvolvimento, e acima de 61%, rico.
# A divisão categórica foi feita a fim de analizar o efeito da situação econômica na mortalidade de pessoas por suicídio.
'''
Legenda:
    Pobre: 3
    Em Desenvolvimento: 2
    Rico: 1
'''
suicidios['classificação econômica'] = np.where((suicidios['gdp_per_capita ($)']/109602)>0.61, 1, 
    (np.where((suicidios['gdp_per_capita ($)']/109602)<0.3, 3,2)))

In [10]:
# criação de uma dummies para armazenar a geração em formato numérico
gen = pd.get_dummies(suicidios['generation'])
suicidios = pd.concat([suicidios, gen], axis = 1)
suicidios.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),...,5-14 years,55-74 years,75+ years,classificação econômica,Boomers,G.I. Generation,Generation X,Generation Z,Millenials,Silent
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",...,0,0,0,3,0,0,1,0,0,0
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",...,0,0,0,3,0,0,0,0,0,1
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",...,0,0,0,3,0,0,1,0,0,0
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",...,0,0,1,3,0,1,0,0,0,0
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",...,0,0,0,3,1,0,0,0,0,0


#Transformação da coluna de geração (generation) de valores string para valores numéricos.
"""
Legenda:

    Boomers - 1

    G.I. Generation - 2

    Generation X - 3

    Generation Z - 4

    Millenials - 5

    Silent - 6

"""
suicidios['generation'] = suicidios['generation'].astype('object') #.astype('int')
#suicidios['generation'] = suidicios['generation'].int64 = [1, 2, 3, 4, 5, 6]
suicidios['generation']

In [11]:
# criação de uma dummies para armazenar o sexo em formato numérico
sexo = pd.get_dummies(suicidios['sex'])
suicidios = pd.concat([suicidios, sexo], axis = 1)
suicidios.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),...,75+ years,classificação econômica,Boomers,G.I. Generation,Generation X,Generation Z,Millenials,Silent,female,male
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",...,0,3,0,0,1,0,0,0,0,1
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",...,0,3,0,0,0,0,0,1,0,1
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",...,0,3,0,0,1,0,0,0,1,0
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",...,1,3,0,1,0,0,0,0,0,1
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",...,0,3,1,0,0,0,0,0,0,1


In [9]:
#Transformação da coluna de sexo (sex) de valores string para valores numéricos.
"""
Legenda:
    male - 2
    female - 1
"""
suicidios['sex'] = suicidios['sex'].astype('int64')
suicidios['sex'].int64 = [1, 2]
#divisão de sexo a fim de comparar depois, se o sexo da pessoa tem relação ao número de suicídios.
sexo_m = suicidios.loc[suicidios['sex'] == 'male',:]
sexo_f = suicidios.loc[suicidios['sex'] == 'female',:]
suicidios.head()

ValueError: invalid literal for int() with base 10: 'male'

In [12]:
suicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  27820 non-null  object 
 1   year                     27820 non-null  int64  
 2   sex                      27820 non-null  object 
 3   age                      27820 non-null  object 
 4   suicides_no              27820 non-null  int64  
 5   population               27820 non-null  int64  
 6   suicides/100k pop        27820 non-null  float64
 7   country-year             27820 non-null  object 
 8   HDI for year             8364 non-null   float64
 9    gdp_for_year ($)        27820 non-null  object 
 10  gdp_per_capita ($)       27820 non-null  int64  
 11  generation               27820 non-null  object 
 12  15-24 years              27820 non-null  uint8  
 13  25-34 years              27820 non-null  uint8  
 14  35-54 years           

# Multiple Regression

Tendo a preparação da base de dados completa, podemos iniciar o estudo da relação das variáveis e assim, prever o número de pessoas a cometer suicídio em um ano.

---
Para iniciar a aplicação, é necessário separar os dados independentes dos dados dependentes.

Para isto, "X" foi declarado como sendo as variáveis independentes, e "y" foi definido como o valor dependente. 

In [34]:
#Regressão linear simples analisando o número de suicídios pelo pib por capita
#scatterplot suicides/ gdp per capita
gdp_pc = suicidios['gdp_per_capita ($)']
num = suicidios['suicides_no']

slope, intercept, r, p, std_err = stats.linregress(num,gdp_pc)

def myfunc(x):
    return slope * num + intercept

modelo = list(map(myfunc, num))

plt.scatter(num,gdp_pc)
plt.plot(num,modelo)
plt.show()

MemoryError: Unable to allocate 217. KiB for an array with shape (27820,) and data type float64

In [14]:
X = suicidios[['15-24 years', '25-34 years', '35-54 years', '55-74 years', '75+ years', 'classificação econômica', 'Boomers', 'G.I. Generation', 'Generation X', 'Generation Z', 'Millenials', 'Silent', 'female', 'male']]
y = suicidios['suicides_no']

In [22]:
regres = linear_model.LinearRegression()
regres_fit = regres.fit(X,y)

In [23]:
print(regres_fit.coef_)

[ 1.42781525e+02  1.74205977e+02  4.22156805e+02  2.58273893e+02
  3.61337915e+01 -1.47920006e+02  5.92569013e+14  5.92569013e+14
  5.92569013e+14  5.92569013e+14  5.92569013e+14  5.92569013e+14
 -3.52241910e+11 -3.52241910e+11]


In [28]:
previsto = regres.predict([[0,1,0,0,0,2,0,0,0,1,0,0,1,0]])
previsto

array([148.75])

In [25]:
score = regres_fit.score(X,y)
print(score)

0.05942516844957313


In [20]:
plt.scatter(X,y)
plt.show()

ValueError: x and y must be the same size

# Bibliografia

_links utilizados:_

- https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/
- https://www.imf.org/en/Publications/WEO/weo-database/2021/April/weo-report?c=512,914,612,614,311,213,911,314,193,122,912,313,419,513,316,913,124,339,638,514,218,963,616,223,516,918,748,618,624,522,622,156,626,628,228,924,233,632,636,634,238,662,960,423,935,128,611,321,243,248,469,253,642,643,939,734,644,819,172,132,646,648,915,134,652,174,328,258,656,654,336,263,268,532,944,176,534,536,429,433,178,436,136,343,158,439,916,664,826,542,967,443,917,544,941,446,666,668,672,946,137,546,674,676,548,556,678,181,867,682,684,273,868,921,948,943,686,688,518,728,836,558,138,196,278,692,694,962,142,449,564,565,283,853,288,293,566,964,182,359,453,968,922,714,862,135,716,456,722,942,718,724,576,936,961,813,726,199,733,184,524,361,362,364,732,366,144,146,463,528,923,738,578,537,742,866,369,744,186,925,869,746,926,466,112,111,298,927,846,299,582,487,474,754,698,&s=NGDPD,PPPGDP,NGDPDPC,PPPPC,&sy=2010&ey=2021&ssm=0&scsm=1&scc=0&ssd=1&ssc=0&sic=1&sort=subject&ds=.&br=1
- https://www.w3schools.com/python/python_ml_multiple_regression.asp
- https://towardsdatascience.com/the-dummys-guide-to-creating-dummy-variables-f21faddb1d40
- 